In [60]:
import time
iss = 0

def make_submit(pred):
    global iss
    submit = pd.DataFrame()
    submit['id'] = test.index
    submit['skilled'] = pred
    iss += 1
    submit.to_csv('submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss), index=False)
    print('last: ', 'submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss))

In [2]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import lightgbm
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import catboost 
import xgboost

__main train__

In [12]:
train = pd.read_csv('data/trian_v2.csv', index_col='id')
test = pd.read_csv('data/test_v2.csv', index_col='id')

In [13]:
add_train = pd.read_csv('data/add_train.csv', index_col='id')
add_train.rename(columns={'0': 'tm_agi_mean', '1': 'tm_attack_range_mean', '2': 'tm_attack_rate_mean ', 
                         '3': 'tm_winrate_mean', '4': 'tm_agi_std', '5': 'tm_attack_range_std', '6': 'tm_attack_rate_std ',
                         '7': 'tm_winrate_std'}, inplace=True)

In [14]:
train = train.drop(['tm_Carry', 'tm_Escape', 'tm_Nuker', 'tm_Initiator', 'tm_Durable', 'tm_Disabler', 'tm_Jungler', 'tm_Support', 'tm_Pusher'], axis=1)
test = test.drop(['tm_Carry', 'tm_Escape', 'tm_Nuker', 'tm_Initiator', 'tm_Durable', 'tm_Disabler', 'tm_Jungler', 'tm_Support', 'tm_Pusher'], axis=1)
train_id_damage_targets = pd.read_csv('json_data/train_id_damage_targets.csv', index_col='Unnamed: 0')
dtc = train_id_damage_targets.columns.tolist()
neutral = []
badguys = []
goodguys = []
for i  in dtc:
    a = i.split('_')
    if 'neutral' in a:
        neutral.append(i)
    if 'badguys' in a:
        badguys.append(i)
    if 'goodguys' in a:
        goodguys.append(i)
        
train['dmt_goodguys'] = train[goodguys].sum(axis=1)
train['dmt_badguys'] = train[badguys].sum(axis=1)
train['dmt_neutral'] = train[neutral].sum(axis=1)

test['dmt_goodguys'] = test[goodguys].sum(axis=1)
test['dmt_badguys'] = test[badguys].sum(axis=1)
test['dmt_neutral'] = test[neutral].sum(axis=1)

In [28]:
target = train.skilled
train = train.drop(['skilled'], axis=1)

In [29]:
X = train
x_test = test #[X.columns]
y = target

In [30]:
clf_1 =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1, max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1, random_state=1) #@- 82 #reg_alpha=10

In [31]:
clf_1.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=12,
        min_child_samples=20, min_child_weight=8, min_split_gain=0.0,
        n_estimators=1500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=1, reg_alpha=9, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [34]:
pred_1 = clf_1.predict(x_test)

__add train__

In [15]:
add_train['dmt_goodguys'] = add_train[goodguys].sum(axis=1)
add_train['dmt_badguys'] =  add_train[badguys].sum(axis=1)
add_train['dmt_neutral'] =  add_train[neutral].sum(axis=1)

In [16]:
add_train = add_train[train.drop(['npc_dota_furion_treant_1',
 'npc_dota_furion_treant_2',
 'npc_dota_furion_treant_3',
 'npc_dota_furion_treant_4',
 'npc_dota_hero_mars',
 'teammate_116_hero',
 'upgrade_behavior empty'], axis=1).columns]

In [17]:
train_1 = train[add_train.columns]

In [18]:
train_1 = train_1.append(add_train)

In [37]:
y_1 = train_1.skilled
train_1 = train_1.drop(['skilled'], axis=1)

In [38]:
clf_2 =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1, max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1, random_state=1) #@- 82 #reg_alpha=10

In [39]:
clf_2.fit(train_1, y_1)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=12,
        min_child_samples=20, min_child_weight=8, min_split_gain=0.0,
        n_estimators=1500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=1, reg_alpha=9, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [40]:
pred_2 = clf_2.predict(x_test[train_1.columns])

__mold__

In [41]:
old_train = pd.read_csv('last_train.csv', index_col='Unnamed: 0')

In [42]:
old_test = pd.read_csv('last_test.csv', index_col='Unnamed: 0')

In [45]:
clf_3 =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1, max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1, random_state=1) #@- 82 #reg_alpha=10

In [46]:
clf_3.fit(old_train, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=12,
        min_child_samples=20, min_child_weight=8, min_split_gain=0.0,
        n_estimators=1500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=1, reg_alpha=9, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [47]:
pred_3 = clf_3.predict(old_test)

# FINAL

In [49]:
pred_1.shape

(43265,)

In [51]:
pred_2.shape

(43265,)

In [52]:
pred_3.shape

(43265,)

In [103]:
random_list = [random.uniform(0, 0.2) for i in range(pred_1.shape[0])]

In [104]:
final = pred_1 + (pred_2 * 0.6) + (pred_3 * 0.2) + random_list + 0.12

In [105]:
final = final / 3

In [106]:
final = (final > 0.5).astype(int)

In [107]:
make_submit(final)

last:  submissions/submission10-7.csv
